# 主题：触觉融合
## 触觉传感器集成到LeapHand灵巧手的ManagerBasedRLEnv任务环境中
我需要在LeapHand灵巧手的ManagerBasedRLEnv任务环境中集成触觉传感器功能，具体需求如下：

**核心需求：**
1. **触觉观测集成**：在观测空间中添加触觉信息，用于策略网络的输入
2. **接触奖励设计**：基于触觉数据设计奖励函数，包括：
   - 正向奖励：适当的接触力度、稳定抓取等
   - 负向惩罚：过度接触力、碰撞等

**技术问题：**
1. **ContactSensor配置**：
   - 如何在环境配置中正确设置ContactSensorCfg？
   > 对资产ArticulationCfg启用activate_contact_sensors=True，这个会对所有link启用ContactReportAPI；但注意配置ContactSensor不会递归包含子刚体，它只关心prim_path指定的刚体
   - 触觉传感器应该附加到哪些body上（指尖、手掌等）？
   > 在InteractiveSceneCfg下对特定prim_path启用ContactSensorCfg，同时要配有filter_prim_paths_expr指定传感器关心的对象
   - 是否可以指定传感器在body上的具体位置和方向？
   > 目前已确认，传感器自动附加到整个body,无法指定body上的局部位置或方向

2. **数据获取与处理**：
   - ContactSensorData中包含哪些可用的触觉信息？
   > 数据类型在 `IsaacLab/source/isaaclab/isaaclab/sensors/contact_sensor/contact_sensor_data.py` 中皆有定义
   > **注意**：`net_forces_w` 和 `force_matrix_w` 的区别在于，前者是所有接触力的矢量和（可能包含未过滤对象），而后者是每个接触点的力矩阵。保险起见，打算优先使用后者
   - 如何获取和处理触觉数据
   > 目前的方法可能是 sensor = env.scene["contact_index"] 然后 data = sensor.data.net_forces_w() 这种
   - 触觉数据的维度和格式是什么？
   > 根据不同数据类型，数据格式有所不同。如pos_w为(N,3)，表示{w}下的传感器原点位置；contact_pos_w为(N,B,M,3)，表示{w}下的接触点位置
   > **注意**：每个 ContactSensor 实例都有自己的一份 ContactSensorData，而文档里形状 (N, B, M, 3) 的第一维 N 指的是“该传感器在并行环境中的实例数（即通常的 num_envs）
   > 即便一个 finger-sensor 对应 1 个被 filter 的 prim，设置 max_contact_data_count_per_prim = 4 表示该 sensor 在整个仿真批次中为每个 (env, body, filter) 最多能记录到最多约 4 个 contact 条目（受全局容量限制）。这些 contact 条目可以非常靠近，因此它们的位置可能几乎重合或非常相似。位置要“代表性点”所以平均；力要“合力”所以求和

3. **实现机制理解**：
   - ContactSensor的底层实现原理（基于PhysX ContactReporter API）
   - 传感器更新频率和数据缓存机制
   - 与其他传感器（如IMU、Camera）的集成方式

4. **具体实现指导**：
   - 在InHandSceneCfg中如何添加ContactSensor配置？
   - 在ObservationsCfg中如何定义触觉观测项？
   > Q：这里便有一个option，究竟是启用二值信号作为输入，还是使用力信号，还要不要使用接触点位置等更丰富的信息。Teacher policy可以使用，但student可能使用二值信号更利于sim2real。目前打算设计1个观察项，根据类型可以使用二值信号或力信号。
   - 在RewardsCfg中如何基于触觉数据设计奖励函数？可参考（`Contact Reward.png`, `Adaptive Reward Curriculum.png`)
   > 关于奖励，一个比较重要的超参数是contact_threshold，排除仿真中的幽灵力（Ghost Forces）。经过 `invest_tactile.py` 测试，设置在 0.1～0.5 之间是比较合适的。
   > 参考 AnyRotate 论文中关于 good contact 和 bad contact的奖励定义，此外设置一个负载分配的奖励，鼓励手指托举而非手心接触物体

### idea
下面记录自己关于 `inhand_tactile_env_cfg.py`(继承自 `inhand_base_env_cfg.py`)的奖励与观察设计的idea

#### 奖励
主要目的是鼓励连续自然、指尖接触密集的手指步态，在 `inhand_base_env_cfg.py`的基础上新增这些奖励项
1. **负载分配奖励**：鼓励手指承担的垂直载荷占总载荷的比例趋近于 1
$$
\begin{aligned}
   r_{load} &= \frac{\sum_{i \in \mathcal{F}} (\mathbf{f}_i \cdot \hat{k})}{\sum_{j \in \mathcal{F} \cup \mathcal{P}} (\mathbf{f}_j \cdot \hat{k}) + \epsilon} \\
   \mathbf{f}_i &= \mathbf{f}_{ni} + \mathbf{f}_{ti}
\end{aligned}
$$
> 说明：$\mathbf{f}_{ni}$ 法向力，来源于 `force_matrix_w`；$\mathbf{f}_{ti}$ 是切向力，来源于 `friction_forces_w`

1. **接触奖励**：鼓励指尖与物体接触形成自然的手指步态
$$
\begin{aligned}
   r_{contact} &= r_{gc} + r_{bc} \\
   r_{gc} &= \begin{cases}
      1 & \text{if } n_{\text{tip-contact}} \ge 2\\
      0 & \text{otherwise}
   \end{cases} \\
   r_{bc} &= \begin{cases}
      1 & \text{if } n_{\text{non-tip-contact}} > 0\\
      0 & \text{otherwise}
   \end{cases}
\end{aligned}
$$

#### 观察
在 `inhand_base_env_cfg.py` 中，观察分为本体感受（关节+动作）、指令（目标位姿/目标位姿差距）、特权信息（物体位姿）。这里是在此基础上，再添加触觉信息。
<br>但 `contact_sensor_data.py` 中提供的信息有许多种。目前采用如下几种触觉类型。
1. **接触力信号**：每个指尖传感器的接触力矢量和
   - 维度：(num_fingers, 3)
   - 说明：表示每个手指指尖与物体接触时的合力，反映了接触力度和方向。这个可以在 teacher policy 里使用。
2. **0-1二值信号**：每个指尖传感器的接触状态
   - 维度：(num_fingers,)
   - 说明：表示每个手指指尖是否与物体接触。这个可以在那个 student policy 里使用。

### 待补
air_time/contact time，friction_forces_w也需要记录？前者不一定作为观察或奖励，而是可以作为训练结果的分析指标；后者是切向力，一定要补充。

### 实验记录
1. `AnyRotate/logs/rl_games/leaphand_object_rot/2026-01-11_22-07-15`
<br>加入触觉奖励和观察后，指尖与物体的接触更丰富，手指步态看上去好力很多，但大拇指的运动仍非常奇怪，整个指尖是偏下去顶物体转，也没有把物体托起来来。

**想法**：
- 接触奖励可以进一步细调？坏接触越多，惩罚越多，好接触越多，奖励越多，而不是单纯的0-1奖励信号？
- 还有一个是从本体感受训练一个策略，然后用这个策略来初始化训练，看看能不能改善大拇指的运动？
- 或者师生在线Dagger，先用本体感受教师策略指导，然后再逐步调低比例，由学生自己摸索
- **重要改进方案**：按照 `AnyRotate` 课程学习（curriculum）逐步打开触觉奖励权重，初期先学“姿态/位姿不崩 + 正常拇指协同”，然后再逐步加大接触相关奖励，避免一开始就掉进“拇指压住就有分”的坑。
**补充**：
关于旋转奖励的设计需要再思考下

2. `AnyRotate/logs/rl_games/leaphand_object_rot/2026-01-12_16-46-33`
<br>按照课程学习逐步打开触觉奖励权重后，还有把之前观察和奖励的触觉阈值对齐之后，手指步态明显改善了很多（而且仅在本体感受观察+历史步为1的情况下，比仅使用本体感受表现更好）

**Trick**：
- 好像课程学习项，rewards mdp本身就能自包含，没必要去设计curriculum mdp

## IsaacLab UI面板加入触觉传感器数据显示
貌似也不用了，主要是一旦设置了和触觉相关的观察项，IsaacLab ManagerBasedRLEnv会自动记录

## 手指步态与接触奖励（可选）
一个更细的想法，借鉴《Sim-to-Real Learning of All Common Bipedal Gaits via Periodic Reward Composition》。
通过一个基于力与速度的周期性奖励框架，鼓励手指形成更丰富的步态模式。而不是当前奖励框架设定下训练所形成的单一接触模式。